# Homework 1

_**[Power Systems Optimization](https://github.com/east-winds/power-systems-optimization)**_

_by Jesse D. Jenkins and Michael R. Davidson (last updated: September 13, 2020)_

This Notebook will walk you through defining a simple transport flow model and then ask you to interact with the solutions and modify to model to add additional constraints...

## Setting up the model

### Load packages

In [1]:
using JuMP
using Clp
using DataFrames
using CSV

### Define sets

We will define two sets, both as arrays of strings

***Production plants, $P$***

In [262]:
P=["trenton", "newark"] # production plants

2-element Array{String,1}:
 "trenton"
 "newark"

***Markets for products, $M$***

In [291]:
M=["newyork", "princeton", "philadelphia"] # markets for products

3-element Array{String,1}:
 "newyork"
 "princeton"
 "philadelphia"

Note that sets can also be defined over intervals (as in `i=1:10`) or numerical vectors (as in `x=[2, 4, 5, 11]`) 

### Define parameters

We'll make use of the defined sets as indexes for our parameters...

***Plant production capacities***

In [264]:
plants = DataFrame(plant=P, capacity=[350,650])

,plant,capacity
,String,Int64
1,trenton,350
2,newark,650


***Demand for products***

Stored in a [DataFrame](https://juliadata.github.io/DataFrames.jl/stable/)

In [292]:
markets = DataFrame(
    market=M, 
    demand=[325, 300, 275]
)

,market,demand
,String,Int64
1,newyork,325
2,princeton,300
3,philadelphia,275


A few different ways to index into our DataFrames to access parameters (all of the below are equivalent)

In [266]:
plants[plants.plant.=="newark",:capacity] # option 1

1-element Array{Int64,1}:
 650

In [267]:
plants[plants.plant.=="newark",:].capacity # option 2

1-element Array{Int64,1}:
 650

In [268]:
plants.capacity[plants.plant.=="newark"] # option 3

1-element Array{Int64,1}:
 650

In [269]:
plants[:,:capacity][plants.plant.=="newark"] # option 4

1-element Array{Int64,1}:
 650

Note that DataFrame indexing returns an Array by default, in this case, a 1-element Array of type Int64 (64-bit integer), as indicated by `Array{Int64,1}` above. 

To access the single Int64 value, append `[1]` to any of the above to reference the first (and only) element in this array. 

In [270]:
plants.capacity[plants.plant.=="newark"][1]

650

In [271]:
typeof(plants.capacity[plants.plant.=="newark"][1])

Int64

In [272]:
typeof(plants.capacity[plants.plant.=="newark"])

Array{Int64,1}

***Distance from plants to markets***

Stored in a JuMP [DenseAxisArray](https://jump.dev/JuMP.jl/v0.19/containers/) with data array and symbolic references across each of our sets (plants and markets), converted to Symbols for referencing

In [274]:
# two dimensional symbolic DenseAxisArray, with from/to distance pairs
distances = JuMP.Containers.DenseAxisArray(
    [2.5 0.5 1.5;
     0.5 1.5 3.5],
    Symbol.(P),
    Symbol.(M),
)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:trenton, :newark]
    Dimension 2, [:newyork, :princeton, :philadelphia]
And data, a 2×3 Array{Float64,2}:
 2.5  0.5  1.5
 0.5  1.5  3.5

A couple example references to our DenseAxisArray to access parameters...

In [218]:
distances[:trenton, :newyork] #example of distance references

2.5

In [219]:
distances[:newark, :newyork] #example of distance references

0.5

In [220]:
distances[Symbol(P[2]),Symbol(M[1])] # another way to find distance from newark to trenton

0.5

In [221]:
distances[Symbol("newark"), Symbol("newyork")] # and a third...

0.5

***Costs of transport***

In [275]:
freight_cost = 90 # Cost of freight shipment per unit of distance

90

### Create model
(and specify the Clp solver)

In [276]:
transport = Model(Clp.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Clp

### Define variables

***Quantities of product to transport from plant $p \in P$ to market $m \in M$***

In [277]:
@variable(transport, X[P,M] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["trenton", "newark"]
    Dimension 2, ["newyork", "princeton", "philadelphia"]
And data, a 2×3 Array{VariableRef,2}:
 X[trenton,newyork]  X[trenton,princeton]  X[trenton,philadelphia]
 X[newark,newyork]   X[newark,princeton]   X[newark,philadelphia]

Example reference to single quantity decision variable, the quantity shipped from Newark to Philadelphia:

In [278]:
X["newark","philadelphia"]

X[newark,philadelphia]

### Define constraints

***Supply capacity constraint***

In [279]:
@constraint(transport, cSupply[p in P], 
    sum(X[p,m] for m in M) 
    <= plants.capacity[plants.plant.==p][1])

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, ["trenton", "newark"]
And data, a 2-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1}:
 cSupply[trenton] : X[trenton,newyork] + X[trenton,princeton] + X[trenton,philadelphia] ≤ 350.0
 cSupply[newark] : X[newark,newyork] + X[newark,princeton] + X[newark,philadelphia] ≤ 650.0

***Demand balance constraint***

Ensure all demand is satisfied at each market

In [280]:
@constraint(transport, cDemand[m in M], 
    sum(X[p,m] for p in P) 
    >= markets.demand[markets.market.==m][1])

1-dimensional DenseAxisArray{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape},1,...} with index sets:
    Dimension 1, ["newyork", "princeton", "philadelphia"]
And data, a 3-element Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.GreaterThan{Float64}},ScalarShape},1}:
 cDemand[newyork] : X[trenton,newyork] + X[newark,newyork] ≥ 325.0
 cDemand[princeton] : X[trenton,princeton] + X[newark,princeton] ≥ 300.0
 cDemand[philadelphia] : X[trenton,philadelphia] + X[newark,philadelphia] ≥ 275.0

### Define objective function

Minimize total cost of transport to satisfy all demand.

First we'll define an expression for total cost of shipments...

In [281]:
@expression(transport, eCost, 
    sum(freight_cost*distances[Symbol(p),Symbol(m)]*X[p,m] 
        for p in P, m in M)
    )

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

Now we'll minimize this total cost

In [282]:
@objective(transport, Min, eCost)

225 X[trenton,newyork] + 45 X[trenton,princeton] + 135 X[trenton,philadelphia] + 45 X[newark,newyork] + 135 X[newark,princeton] + 315 X[newark,philadelphia]

## Interact with the model

**(a)** Now let's solve the model. In the blank cell below, enter the command for JuMP to solve a model and run the cell

**(b)** You've got a solution. Now query the objective function in the empty cell below and save it to a variable (name of your choice)

**(c)** Now query and save the optimal solution for X (the decisions about shipment quantities from plant to market) to an Array or DataFrame

**(d)** Please interpret your results by writing an explanation in the markdown cell below. 

Which facility or facilities supplies the most demand in New York? Does this result make sense? Why?

Which facility or facilities supplies the most demand in Philadelphia? Does this result make sense? Why?

Which facility or facilities supplies the demand in Princeton? Does this result make sense? Why?

**(d)** A new market in New Brunswick appears, with a demand for 50 units. It is located 1.0 units away from both plants. Add this market to the model and solve again.

**(e)** What is new optimal solution? 

**(f)** Interpret this result in the markdown cell below. Which facility or facilities supplies the demand in New Brunswick? Does this result make sense? Why?